In [19]:
import cirq
import qsimcirq
import numpy as np

In [20]:
def print_state(v):
    for i, amp in enumerate(v.final_state_vector):
        print(f"|{i:3d}| {amp.real:.6f}{amp.imag:+.6f}j")

In [21]:
def one():
    qubits = cirq.LineQubit.range(1)
    circuit = cirq.Circuit()

    circuit.append(cirq.H(qubits[0]))
    circuit.append(cirq.X(qubits[0]))
    circuit.append(cirq.Z(qubits[0]))
    circuit.append(cirq.Y(qubits[0]))
    circuit.append(cirq.I(qubits[0]))

    return circuit

In [22]:
def two():
    qubits = cirq.LineQubit.range(2)
    circuit = cirq.Circuit()

    circuit.append(cirq.H(qubits[0]))
    circuit.append(cirq.CX(qubits[0], qubits[1]))
    circuit.append(cirq.Y(qubits[0]))
    circuit.append(cirq.X(qubits[1]))
    circuit.append(cirq.SWAP(qubits[0], qubits[1]))
    circuit.append(cirq.I(qubits[0]))
    circuit.append(cirq.Z(qubits[1]))

    return circuit

In [23]:
def three():
    qubits = cirq.LineQubit.range(3)
    circuit = cirq.Circuit()
    
    circuit.append(cirq.H(qubits[0]))
    circuit.append(cirq.I(qubits[1]))
    circuit.append(cirq.I(qubits[2]))
    circuit.append(cirq.CX(qubits[0], qubits[1]))
    circuit.append(cirq.I(qubits[2]))
    circuit.append(cirq.Y(qubits[0]))
    circuit.append(cirq.CX(qubits[1], qubits[2]))
    circuit.append(cirq.I(qubits[0]))
    circuit.append(cirq.Z(qubits[1]))
    circuit.append(cirq.X(qubits[2]))
    circuit.append(cirq.SWAP(qubits[0], qubits[1]))
    circuit.append(cirq.I(qubits[2]))
    circuit.append(cirq.H(qubits[0]))
    circuit.append(cirq.H(qubits[1]))
    circuit.append(cirq.H(qubits[2]))
    circuit.append(cirq.CX(qubits[2], qubits[0]))
    circuit.append(cirq.X(qubits[0]))
    circuit.append(cirq.Y(qubits[1]))
    circuit.append(cirq.Z(qubits[2]))

    return circuit

In [24]:
def grover(n = 3, mark = 3):
    qubits = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    mcz = cirq.Z.controlled(n - 1)

    circuit.append(cirq.H.on_each(qubits))

    for i, qubit in enumerate(qubits):
        if ((mark >> i) & 1) == 0:
            circuit.append(cirq.X(qubit))

    circuit.append(mcz(*qubits))

    for i, qubit in enumerate(qubits):
        if ((mark >> i) & 1) == 0:
            circuit.append(cirq.X(qubit))

    circuit.append(cirq.H.on_each(qubits))
    circuit.append(cirq.X.on_each(qubits))

    circuit.append(mcz(*qubits))
    
    circuit.append(cirq.X.on_each(qubits))
    circuit.append(cirq.H.on_each(qubits))

    return circuit


In [25]:
def qft(n = 3):
    qubits = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()

    for i in range(n):
        circuit.append(cirq.H(qubits[i]))
        for j in range(i + 1, n):
            phase = j - i + 1
            gate = cirq.CZPowGate(exponent=phase)
            circuit.append(gate(qubits[j], qubits[i]))

    for i in range (n // 2):
        circuit.append(cirq.SWAP(qubits[i], qubits[n - 1 - i]))

    return circuit

In [26]:
def default_qft(n = 3):
    qubits = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()

    circuit.append(cirq.qft(*qubits))

    return circuit

In [27]:
import time

def benchmark_circuits(circuits, num_runs=10, warmup_runs=2, simulator = None):
    results = {}

    for name, circuit in circuits.items():
        print(f"\nbenchmarking {name}")
        
        for _ in range(warmup_runs):
            simulator.simulate(circuit)
        
        time_data = []
        for _ in range(num_runs):
            start_time = time.time()
            simulator.simulate(circuit)
            end_time = time.time()
            time_data.append(end_time - start_time)
        
        avg_time = np.mean(time_data)
        std_time = np.std(time_data)
        results[name] = {'average_time': avg_time, 'std_dev': std_time}
        print(f"{name}: average time = {avg_time:.4f} seconds, std dev = {std_time:.4f} seconds")

    return results

In [ ]:
simulator = qsimcirq.QSimSimulator()

one_circuit = one()
two_circuit = two()
three_circuit = three()
qft_circuit = qft(26)
grover_circuit = grover(26, 0b0)

circuits = {
    "one": one_circuit,
    "two": two_circuit,
    "three": three_circuit,
    "grover": grover_circuit,
    "qft": qft_circuit,
}

results = benchmark_circuits(circuits, 10, 2, simulator=simulator)

#print(grover_circuit)
#print(qft_circuit)


benchmarking one
one: average time = 0.0001 seconds, std dev = 0.0000 seconds

benchmarking two
two: average time = 0.0001 seconds, std dev = 0.0000 seconds

benchmarking three
three: average time = 0.0003 seconds, std dev = 0.0000 seconds

benchmarking grover
